In [ ]:
job_worker_name = "g_b3_33"
!pip uninstall -y fashion_nets
!pip uninstall -y fashion_datasets

In [ ]:
#@title Pip Installs & Imports
#!pip uninstall -y fashion_nets
!pip install -q git+https://github.com/NiklasHoltmeyer/FashionNets.git
try: 
  from fashionnets.train_jobs.loader.job_loader import prepare_environment, load_job_settings, history_to_csv_string
  from fashionnets.train_jobs.loader.model_loader import load_siamese_model_from_train_job
except:
  from fashionnets.train_jobs.loader.job_loader import prepare_environment, load_job_settings, history_to_csv_string
  from fashionnets.train_jobs.loader.model_loader import load_siamese_model_from_train_job

In [ ]:
#@title Prepare Environment. Download Datasets
from fashionnets.train_jobs.loader.job_loader import add_back_bone_to_train_job

environment, training_job_cfg = prepare_environment(job_worker_name, debugging=False)
import kaggle #<- Requires Secrets, therefore prepare_environment needs to be run before
kaggle.api.authenticate()

kaggle_downloader = kaggle.api.dataset_download_files #<- sadly needs to be injected

train_job = load_job_settings(environment=environment, training_job_cfg=training_job_cfg, kaggle_downloader=kaggle_downloader)

job_settings = add_back_bone_to_train_job(environment=environment, **training_job_cfg)

In [ ]:
siamese_model, init_epoch, _callbacks = load_siamese_model_from_train_job(load_weights=True, **train_job)
print("init_epoch", init_epoch)
print("Callbacks", _callbacks)

In [ ]:
from fashionnets.train_jobs.loader.dataset_loader import build_dataset_hard_pairs_deep_fashion
import tensorflow as tf

historys = []
build_frequency = 5

for idx in range(train_job["epochs"] // build_frequency):
    init_epoch_local = idx * build_frequency
    epochs = init_epoch_local + build_frequency
    
    if init_epoch_local < init_epoch:
        continue

    print(init_epoch_local)
    
    dataset = build_dataset_hard_pairs_deep_fashion(siamese_model, job_settings)

    history = siamese_model.fit(dataset["train"], 
                            epochs=epochs, 
                            validation_data=dataset["val"], 
                            callbacks = _callbacks,
                            initial_epoch=init_epoch) 
    historys.append(history)
    

In [ ]:
_callbacks[-1].on_epoch_end(100000, None) #<- force reupload all Zips